<a href="https://colab.research.google.com/github/LorenzoBellomo/InformationRetrieval/blob/main/notebooks/5_GraphDBs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GraphDBs and Neo4j

In this lecture, we focus on GraphDBs, using Neo4j and its language (Cypher) to make some simple queries.

In [1]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 5.0 MB/s eta 0:00:00


In [2]:
!pip install yfiles_jupyter_graphs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [3]:
from neo4j import GraphDatabase

Go to the neo4j sandbox and click on the box "blank project" ([https://sandbox.neo4j.com/](https://sandbox.neo4j.com/)). After the sandbox has been created, click on the rightmost black arrow, click on "connections details", and copy the BOLT URL and password in the command just below

In [20]:
driver = GraphDatabase.driver(<INSERT BOLT URL>, auth=("neo4j", <INSERT PSW>))
session = driver.session(database="neo4j")
driver.execute_query("MATCH (n) DETACH DELETE n  ") # purging eventual graphs already loaded

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7acc583d1c50>, keys=[])

Now we are ready to populate the graph

In [21]:
driver.execute_query(
    "CREATE (john:Person {name: \"John\"})   \
    CREATE (joe:Person {name: \"Joe\"})      \
    CREATE (steve:Person {name: \"Steve\"})  \
    CREATE (sara:Person {name: \"Sara\"})    \
    CREATE (maria:Person {name: \"Maria\"})  \
    CREATE (john)-[:FRIEND]->(joe)-[:FRIEND]->(steve)    \
    CREATE (john)-[:FRIEND]->(sara)-[:FRIEND]->(maria)   \
")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7acc583f1fd0>, keys=[])

In [13]:
# we execute only one query, but the user could specify several queries as a list of strings # TODO CONTROLLARE
results = driver.execute_query("MATCH (n) return n") # n is reserved to access nodes

Let's print the results, results are a list of "node entries". To access them we need to use the "name" of the variable that we used in the query (n)

In [14]:
for record in results[0]: # We only run 1 query, so the response are in results[0]
    # iterating over all nodes
    elem = record["n"]
    print("{} node with name {}".format(list(elem.labels)[0], elem["name"])) # neo4j supports multiple labels (types) per node and edge

Person node with name John
Person node with name Joe
Person node with name Steve
Person node with name Sara
Person node with name Maria


Let's use this widget library to see the graph visually (for ease of understanding)

In [15]:
from yfiles_jupyter_graphs import GraphWidget
from google.colab import output
output.enable_custom_widget_manager()
def draw_graph():
    g = session.run("MATCH (n) -[e]-> (m) return n, m, e").graph()
    gw = GraphWidget(graph=g)
    return gw

draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

Now we try to add a new edge from nodes "Sara" and "Joe"

In [16]:
results = driver.execute_query("CREATE (j:Person {name: \"Sara\"})-[rel:FRIEND]->(m:Person {name: \"Joe\"})  ")
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

Oops, we meessed up, we did not "bind" the two names to the nodes already present. First, let's delete the two new nodes we created.

In [17]:
results = (driver.execute_query("MATCH (n {name: \"Sara\"}) -[]-> (m {name: \"Joe\"}) RETURN n, m"))
res = results[0][0]
print(res)
print(res["n"].element_id)
print(res["m"].element_id)
driver.execute_query("MATCH (n) where elementId(n)=$id DETACH DELETE n  ", id=res["n"].element_id)
driver.execute_query("MATCH (n) where elementId(n)=$id DETACH DELETE n  ", id=res["m"].element_id)
draw_graph()

<Record n=<Node element_id='4:1584a58c-6298-4af9-ac3f-2dab48b8bfb1:5' labels=frozenset({'Person'}) properties={'name': 'Sara'}> m=<Node element_id='4:1584a58c-6298-4af9-ac3f-2dab48b8bfb1:6' labels=frozenset({'Person'}) properties={'name': 'Joe'}>>
4:1584a58c-6298-4af9-ac3f-2dab48b8bfb1:5
4:1584a58c-6298-4af9-ac3f-2dab48b8bfb1:6


GraphWidget(layout=Layout(height='500px', width='100%'))

To actually add the edge we wanted to add, we need to use the "MERGE" statement

In [22]:
driver.execute_query("MATCH (j:Person {name: \"Sara\"})  MATCH (m:Person {name: \"Joe\"})  MERGE (j)-[r:FRIEND]->(m)")
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

Let's now add two countries, and let's connect our nodes to those countries

In [23]:
driver.execute_query("CREATE (:Country {name: \"Australia\"})  CREATE (:Country {name: \"Germany\"}) ")
driver.execute_query("MATCH (j:Person {name: \"Sara\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l) ")
driver.execute_query("MATCH (j:Person {name: \"Maria\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l) ")
driver.execute_query("MATCH (j:Person {name: \"John\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l) ")
driver.execute_query("MATCH (j:Person {name: \"Joe\"})  MATCH (l:Country {name: \"Australia\"})  MERGE (j)-[r:LIVES_IN]->(l) ")
driver.execute_query("MATCH (j:Person {name: \"Steve\"})  MATCH (l:Country {name: \"Australia\"})  MERGE (j)-[r:LIVES_IN]->(l) ")
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

Testing regex in the query

In [24]:
results = driver.execute_query("MATCH (p:Person) WHERE p.name =~ 'Jo.*' RETURN p.name  ")
for res in results[0]:
    print(res["p.name"])

John
Joe


Testing the "exists" keyword

In [25]:
results = driver.execute_query("MATCH (p:Person)-[r:FRIEND]->(m:Person)  WHERE exists((m)-[:FRIEND]->(p))  RETURN m, p ")
for res in results[0]:
    print(res["m"]["name"], " -> ", res["p"]["name"])

aggregate and count

In [ ]:
# we use the keyword "as" to name the adjacency list (retrieved with collect) and its size (retrieved with count)
results = driver.execute_query("MATCH (p1:Person)-[:FRIEND]->(p2:Person) RETURN p1.name, collect(p2.name) as listOfFriends, count(*) as numberOfFriends")
for res in results[0]:
  print("{} has {} friends {}".format(res["p1.name"], res["numberOfFriends"], res["listOfFriends"]))

John has 2 friends ['Joe', 'Sara']
Joe has 1 friends ['Steve']
Sara has 2 friends ['Joe', 'Maria']


**EX1:** Count the number of people who have at least one friend that lives in Australia

**EX2:** Print the names of people who have at least one friend that lives in Australia

In [ ]:
q = "TODO - ADD QUERY HERE "
results = driver.execute_query(q)
for res in results[0]:
    print(res)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'TODO': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"TODO - ADD QUERY HERE"
 ^}

Let's now check how directed edges work. This next query uses unidirect edges (-[related_to]-).

Try changing it to directed ones (-[related_to]->).

In [28]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[y]-(x)  return x.name, type(y), Labels(x)[0]")
for res in results[0]:
  print("Maria -[{}]-> {} ({})".format(res["type(y)"], res["x.name"], res["Labels(x)[0]"]))

Maria -[FRIEND]-> Sara (Person)
Maria -[LIVES_IN]-> Germany (Country)


indirect paths of max length 2 from Maria

In [29]:
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [ ]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[*1..2]-(x)  return DISTINCT x ")
for res in results[0]:
  print(res["x"]["name"])

Sara
Germany
Joe
John


direct paths of max length 2 from Maria

In [ ]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[*1..2]->(x)  return DISTINCT x ")
for res in results[0]:
  print(res["x"]["name"])

Germany


Check the database schema, to see which kind of edges connect which kinds of nodes

In [30]:
g = session.run("CALL db.schema.visualization()").graph()
GraphWidget(graph=g)

GraphWidget(layout=Layout(height='500px', width='100%'))

To run large scale data science algorithms, we need to create an in-memory projection of our graph

In [32]:
# "in_memory_graph" is the name we assign to the RAM projection of our graph
results = driver.execute_query("CALL gds.graph.project(\'in_memory_graph\', \
    [\'Person\', \'Country\'], [\'FRIEND\', \'LIVES_IN\'])  \
    YIELD graphName AS graph, nodeProjection, nodeCount AS nodes, relationshipCount AS rels  ")
print(results[0])

[<Record graph='in_memory_graph' nodeProjection={'Country': {'label': 'Country', 'properties': {}}, 'Person': {'label': 'Person', 'properties': {}}} nodes=7 rels=10>]


let's compute the page rank of all nodes. Higher pagerank nodes are "more important" ones.

In [36]:
results = driver.execute_query("""CALL gds.pageRank.stream(\'in_memory_graph\')
    YIELD nodeId, score WITH gds.util.asNode(nodeId) AS n,score AS pageRank
    RETURN n.name AS name, Labels(n)[0] AS type, pageRank ORDER BY pageRank DESC """)
for res in results[0]:
  # sorted by decreasing page rank
  print(res["name"], " - ", res["type"], " - ", round(res["pageRank"]*100)/100)

Australia  -  Country  -  0.47
Germany  -  Country  -  0.42
Steve  -  Person  -  0.25
Joe  -  Person  -  0.25
Maria  -  Person  -  0.2
Sara  -  Person  -  0.19
John  -  Person  -  0.15


Let's re-compute the pagerank of the nodes, but let's also write those values as a node property.

In [38]:
results = driver.execute_query("CALL gds.pageRank.write('in_memory_graph',     {writeProperty: 'pageRank'} ) YIELD nodePropertiesWritten, ranIterations ")
results = driver.execute_query("MATCH (n) RETURN n.name as name, n.pageRank as pr ORDER BY pr DESC")
for res in results[0]:
  print(res["name"], " - ", round(res["pr"]*100)/100)

Australia  -  0.47
Germany  -  0.42
Steve  -  0.25
Joe  -  0.25
Maria  -  0.2
Sara  -  0.19
John  -  0.15


Let's also compute the shortest path between John and Maria

In [43]:
results = driver.execute_query("""MATCH (source:Person {name: \'John\'}) MATCH (target:Person {name: \'Maria\'})
    CALL gds.shortestPath.dijkstra.stream(\'in_memory_graph\', {    sourceNode: source,
    targetNode: target}) YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN    index, totalCost, [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs ORDER BY index""")

for res in results[0]:
    print("PATH is {} (its incremental cost is {}, for a total cost of {})".format(res["nodeNames"], res["costs"], res["totalCost"]))

PATH is ['John', 'Sara', 'Maria'] (its incremental cost is [0.0, 1.0, 2.0], for a total cost of 2.0)


Let's also export the graph in CSV. This is to write it to an output stream.

In [ ]:
driver.execute_query("CALL apoc.export.csv.all(null, {stream:TRUE})")

EagerResult(records=[<Record file=None source='database: nodes(7), rels(10)' format='csv' nodes=7 relationships=10 properties=14 time=15 rows=17 batchSize=20000 batches=1 done=True data='"_id","_labels","name","pageRank","_start","_end","_type"\n"0",":Person","John","0.15000000000000002",,,\n"1",":Person","Joe","0.24704166666666671",,,\n"2",":Person","Steve","0.2549927083333334",,,\n"3",":Person","Sara","0.19250000000000003",,,\n"4",":Person","Maria","0.2045416666666667",,,\n"5",":Country","Australia","0.47173651041666675",,,\n"6",":Country","Germany","0.4209020833333334",,,\n,,,,"0","1","FRIEND"\n,,,,"0","3","FRIEND"\n,,,,"0","6","LIVES_IN"\n,,,,"1","2","FRIEND"\n,,,,"1","5","LIVES_IN"\n,,,,"2","5","LIVES_IN"\n,,,,"3","1","FRIEND"\n,,,,"3","4","FRIEND"\n,,,,"3","6","LIVES_IN"\n,,,,"4","6","LIVES_IN"\n'>], summary=<neo4j._work.summary.ResultSummary object at 0x79c9e741b850>, keys=['file', 'source', 'format', 'nodes', 'relationships', 'properties', 'time', 'rows', 'batchSize', 'batches'

Let's purge the graph, to load another one from two csvs.

In [44]:
driver.execute_query("MATCH (n) DETACH DELETE n ")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7acc51f22c90>, keys=[])

here we load the nodes and edges csvs we put in the repository.

In [45]:
driver.execute_query("LOAD CSV WITH HEADERS FROM \"https://raw.githubusercontent.com/LorenzoBellomo/InformationRetrieval/main/data/nodes.csv\" AS line CREATE (:Person {name: line.name, lastName: line.last_name, my_id: line._id}) ")
driver.execute_query("LOAD CSV WITH HEADERS FROM \"https://raw.githubusercontent.com/LorenzoBellomo/InformationRetrieval/main/data/edges.csv\" AS line MATCH (p1:Person {my_id: line.source}) MATCH (p2:Person {my_id: line.target})  MERGE (p1) –[:SUPERVISOR]->(p2) ")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7acc51ff4b90>, keys=[])

let's visualize the graph

In [46]:
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

## New exercises
Let's create a new sandbox project (Movies). Let's also connect to that instance, and do the proposed exercises.